In [5]:
from main import *

warnings.simplefilter('ignore')

model_path: Optional[Path] = Path('../models/latest.ckpt')
onnx_model_path: Path = Path('../models/model.onnx')
data_path: Path = Path('../data')
num_simulation: int = 500
num_iter: int = 100
num_worker = os.cpu_count()
print(data_path)
module = LightingModule()
policy = np.load(data_path / 'policy.npy')
states = np.load(data_path / 'states.npy').astype(np.float32)
values = np.load(data_path / 'values.npy').astype(np.float32)
train_p, val_p, train_s, val_s, train_v, val_v = train_test_split(
    policy, states, values, test_size=0.2, shuffle=True, random_state=42
)
train_dataset = Dataset(train_s, train_p, train_v)
val_dataset = Dataset(val_s, val_p, val_v)
train_dataloder = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloder = DataLoader(val_dataset, batch_size=256)
trainer = pl.Trainer(
    min_epochs=10,
    max_epochs=100,
    log_every_n_steps=10,
    logger=[],
    callbacks=[
        EarlyStopping(monitor='val_loss'),
    ],
    checkpoint_callback=False,
)
trainer.fit(module, train_dataloder, val_dataloder)
trainer.save_checkpoint(model_path)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name   | Type              | Params
---------------------------------------------
0 | model  | Model             | 288 K 
1 | loss_p | BCEWithLogitsLoss | 0     
2 | loss_v | MSELoss           | 0     
---------------------------------------------
288 K     Trainable params
0         Non-trainable params
288 K     Total params
1.154     Total estimated model params size (MB)


../data
Epoch 23: 100%|██████████| 233/233 [00:04<00:00, 49.33it/s, loss=1.13, v_num=]
